In [1]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_path = "../model_exports/fine_tuned_bert"

model = TFAutoModelForSequenceClassification.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)


2024-11-25 22:49:33.966487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732571374.277272   38066 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732571374.364355   38066 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 22:49:35.031014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1732571381.557344   38066 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/clean/dataset.csv").fillna('')
X = df["title"]
y = df["funny"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Prepare dataset
import tensorflow as tf
def preprocess_function(sentences, labels, tokenizer, max_length=128):
    inputs = tokenizer(sentences, truncation=True, padding=True, max_length=max_length, return_tensors="tf")
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), labels))
    return dataset.batch(8)

In [7]:
test_dataset = preprocess_function(list(X_test), list(y_test), tokenizer)

In [9]:
# Generate predictions
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Evaluate with additional metrics
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_labels, target_names=["Class 0", "Class 1"]))

2665/2665 [==============================] - 490s 181ms/step
              precision    recall  f1-score   support

     Class 0       0.96      0.99      0.98     17359
     Class 1       0.94      0.84      0.89      3954

    accuracy                           0.96     21313
   macro avg       0.95      0.91      0.93     21313
weighted avg       0.96      0.96      0.96     21313



In [16]:
from scipy.special import softmax
# Convert logits to probabilities using softmax
probas = softmax(predictions.logits, axis=1)

results_df = pd.DataFrame({
    "y_true": y_test,
    "y_pred": predicted_labels,
    "proba": probas[:,1],
    "title": X_test, 
})

false_positives = results_df[(results_df["y_true"] == 0) & (results_df["y_pred"] == 1)]
true_positives = results_df[(results_df["y_true"] == 1) & (results_df["y_pred"] == 1)]
false_negatives = results_df[(results_df["y_true"] == 1) & (results_df["y_pred"] == 0)]
true_negatives = results_df[(results_df["y_true"] == 0) & (results_df["y_pred"] == 0)]

sorted_false_positives = false_positives.sort_values(by="proba", ascending=False)
sorted_true_positives = true_positives.sort_values(by="proba", ascending=False)
sorted_false_negatives = false_negatives.sort_values(by="proba", ascending=False)
sorted_false_negatives_asc = false_negatives.sort_values(by="proba", ascending=True)
sorted_true_negatives = true_negatives.sort_values(by="proba", ascending=False)


print(sorted_false_positives.head())
print(sorted_true_positives.head())
print(sorted_false_negatives.head())
print(sorted_true_negatives.head())

       y_true  y_pred     proba  \
95199       0       1  0.998561   
23559       0       1  0.998519   
53181       0       1  0.998206   
94885       0       1  0.998069   
92461       0       1  0.997829   

                                                   title  
95199  VIDÉO. L'homme qui murmure à l'oreille des c...  
23559  VIDEO. Une "trombe marine" se transforme en to...  
53181  Il est désormais possible de parier sur le rés...  
94885  Ces ralentisseurs sont illégaux, on vous dit ...  
92461  Elle disait vouloir « manger des enfants et le...  
       y_true  y_pred     proba  \
28206       1       1  0.998979   
33876       1       1  0.998978   
28383       1       1  0.998978   
50252       1       1  0.998977   
28953       1       1  0.998977   

                                                   title  
28206  457 guitaristes jouent "Highway to Hell" d'AC/...  
33876  VIDÉO. Les fenêtres à bascule de ce gratte-...  
28383  Ce raton laveur (ou plutôt escaladeur) 

In [17]:
for index, row in sorted_false_positives.head(100).iterrows():
    print(row['title'], row['proba'])

VIDÉO. L'homme qui murmure à l'oreille des cachalots 0.998561441898346
VIDEO. Une "trombe marine" se transforme en tornade et ravage des champs de tournesols 0.9985188841819763
Il est désormais possible de parier sur le résultat de l'élection présidentielle américaine 0.9982055425643921
Ces ralentisseurs sont illégaux, on vous dit pourquoi 0.9980688691139221
Elle disait vouloir « manger des enfants et les dessiner nus » sur le tchat en ligne de la police 0.9978287816047668
Des parfums qui ont du chien 0.9975473284721375
Joe Biden: un honnête homme s'en va 0.9973794221878052
Aux États-Unis, des policiers interviennent pour dégager une citrouille géante gonflable de la route 0.996993899345398
Le premier qui dit la vérité se fait étriller 0.9966431856155396
Un cheval remporte une course avec une cote colossale... mais meurt juste après 0.9962510466575623
Royaume-Uni : pour son anniversaire, une centenaire s'offre un saut en parachute 0.996246874332428
L'Euro de foot des sans-abri 0

In [18]:
for index, row in sorted_true_positives.head(100).iterrows():
    print(row['title'], row['proba'])

457 guitaristes jouent "Highway to Hell" d'AC/DC en même temps et ça fait beaucoup de bruit 0.998979389667511
VIDÉO. Les fenêtres à bascule de ce gratte-ciel de Chicago vont vous donner le vertige 0.9989783763885498
Ce raton laveur (ou plutôt escaladeur) n'a rien à envier à Spider-Man 0.9989781379699707
La réaction de ce couple d'agriculteurs en apprenant qu'il a gagné au Loto va vous émouvoir 0.9989774227142334
Ces Coréens rois du beatbox reprennent "Despacito" et c'est parfait 0.9989768266677856
VIDÉO. La police néo-zélandaise recrute et elle l'a fait savoir d'une drôle de manière 0.9989707469940186
Ce pilote d'avion est d'une franchise confondante 0.9989686012268066
En mangeant 75 hot-dogs d'affilé, cet Américain a battu son propre record du monde 0.9989684820175171
VIDEO. Ce petit garçon qui tente de casser une planche au Taekwondo va vous faire fondre 0.9989681243896484
VIDÉO. Un homme essaie de sabrer en deux une pastèque et c'est une mauvaise idée 0.998967647

In [19]:
for index, row in sorted_false_negatives.head(100).iterrows():
    print(row['title'], row['proba'])

Non, le roquefort n’est pas « un remède contre le Covid-19 » 0.4991614818572998
Les Caennais oublient un joueur à Strasbourg après leur match nul 0.4981422424316406
Le Petit Robert se trompe dans la définition de la truffade, levée de boucliers en Auvergne 0.4974924623966217
Comment utiliser le marc de café ? Avec ces 8 astuces, vous ne le jetterez plus 0.4960711598396301
Paris : une locataire transforme ses WC en douche et se fait expulser de son appartement 0.4957179129123688
Un Chinois arrêté pour avoir payé 200 faux invités à son mariage 0.49571728706359863
Burger King chambre Arras... et se fait chambrer à son tour 0.49465858936309814
Au Chili, les habitants d'une ville se mobilisent pour sauver un éléphant de mer 0.4841061532497406
VIDEO. 40 auditions de stars avant qu'elles ne deviennent connues 0.4797017574310303
Les Sud-coréens vont rajeunir d’un an ou deux, c’est la loi qui le dit 0.4797000586986542
« C'est un métier de dingue »: à 16 ans, Julien a été mast

In [20]:
for index, row in sorted_false_negatives_asc.head(100).iterrows():
    print(row['title'], row['proba'])

Contrefaçon : 4 000 faux CD de la chanteuse Lio et d'autres artistes saisis dans le Gard 0.0003723681438714266
227 tests d’alcoolémie effectués durant le week-end en Pays de Brest : zéro résultat positif 0.00037464016349986196
Yseult aux Vieilles Charrues : « Bonsoir La Rochelle ! » 0.00037729571340605617
Les corps de cinq Poilus identifiés, 106 ans après leur mort : "Une surprise brutale" 0.00037920413888059556
Elle vendait plusieurs fois ses bébés : une mère porteuse condamnée pour escroquerie à Blois 0.00038444032543338835
"La première chose, c'est d'évaluer le danger" : un caïman capturé par les pompiers près de Toulouse 0.0003867048944812268
Les Archives du Père Castor inscrites à l’inventaire « Mémoire du monde » de l’UNESCO 0.0003881127049680799
Des indépendantistes bretons appellent au boycott de Bécassine 0.00038977141957730055
Rencontre historique à Cuba entre le pape et le patriarche russe Kirill 0.00039630025275982916
Élections. En Corée du Nord, le parti a

In [21]:
for index, row in sorted_true_negatives.head(100).iterrows():
    print(row['title'], row['proba'])

Argentine : des perroquets délogés de leur habitat naturel font vivre un enfer aux habitants de la province de Buenos Aires 0.4979465901851654
Une entreprise suisse lance de l’eau de Paris en bouteille 0.494498074054718
Serge Delfau présente sa "grande marée" 0.4854878783226013
Le bœuf de Matsusaka, fondant comme du foie gras 0.48446816205978394
Dans le pays de Retz, la mort de huit moutons fait polémique 0.4728262722492218
VIDÉO. Céline Dion fait le show pour la promotion d’un match de NFL 0.4651547372341156
Pour une pizza trop chère, il frappe le pizzaïolo puis les policiers : "je suis parti au quart de tour" 0.46444886922836304
A la Mostra de Venise, Daniel Craig, en intello gay, se perd dans le caricatural « Queer » 0.4597037434577942
Ivre, l’étudiant force l’entrée d’un fast-food pour se rendre aux toilettes et s’endort à l’intérieur 0.4548991322517395
Le ginkgo biloba, survivant d'Hiroshima 0.45155513286590576
Le "trou de tête", un danger méconnu du breakdance 0.449340611696